In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf

import tensorflow_addons as tfa
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from keras.utils import pad_sequences

In [2]:
df=pd.read_csv('input9.csv')

In [3]:
df.head()

,comment,class
0,FIXME formatters are not thread safe,DEFECT
1,"XXX Jon Skeet The comment ""if it hasn't bee...",DEFECT
2,"I hate to admit it, but we don't know what h...",DEFECT
3,Just a note StarTeam has a status for NEW wh...,DEFECT
4,the generated classes must not be added in t...,DEFECT


In [4]:
df['class'].value_counts()

WITHOUT_CLASSIFICATION    58204
DESIGN                     2703
IMPLEMENTATION              757
DEFECT                      472
TEST                         85
DOCUMENTATION                54
Name: class, dtype: int64

In [5]:
df_satd = df[(df['class']=='DESIGN') | (df['class']== 'IMPLEMENTATION' ) | (df['class']== 'DEFECT') | (df['class']== 'TEST') | (df['class']== 'DOCUMENTATION') ]


In [6]:
df_nsatd = df[df['class']=='WITHOUT_CLASSIFICATION']

In [7]:

df_nsatd_downsampled = df_nsatd.sample(df_satd.shape[0])

In [8]:
df_balanced = pd.concat([df_nsatd_downsampled, df_satd])

In [9]:
df_balanced['class'].value_counts()

WITHOUT_CLASSIFICATION    4071
DESIGN                    2703
IMPLEMENTATION             757
DEFECT                     472
TEST                        85
DOCUMENTATION               54
Name: class, dtype: int64

In [10]:
df_balanced['spam']=df_balanced['class'].apply(lambda x: 0 if x=='WITHOUT_CLASSIFICATION' else 1)

In [11]:
df_balanced['spam'].value_counts()

0    4071
1    4071
Name: spam, dtype: int64

In [12]:
df_balanced.head()

,comment,class,spam
9975,$NON NLS 1$,WITHOUT_CLASSIFICATION,0
50124,Remove all the series from the collection an...,WITHOUT_CLASSIFICATION,0
25432,Open the output file,WITHOUT_CLASSIFICATION,0
28810,Map to an eClass,WITHOUT_CLASSIFICATION,0
35722,@bgen jjtree ReturnStatement,WITHOUT_CLASSIFICATION,0


In [13]:
X = df_balanced['comment']
Y = df_balanced['spam']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, stratify = Y, random_state=42)

In [15]:
max_words = 10000
max_len = 250
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [16]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(128)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [30]:
model = RNN()
model.summary()
opt = tfa.optimizers.AdaBelief(learning_rate=5e-3)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 250)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 250, 50)           500000    
                                                                 
 lstm_3 (LSTM)               (None, 128)               91648     
                                                                 
 FC1 (Dense)                 (None, 256)               33024     
                                                                 
 activation_6 (Activation)   (None, 256)               0         
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [31]:
#model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_loss',
                mode='min',
                save_best_only=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001)

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=50,validation_split=0.25, callbacks=[model_checkpoint_callback,reduce_lr])

model.load_weights(checkpoint_filepath)

Epoch 1/50
39/39 [==============================] - 4s 48ms/step - loss: 0.6579 - accuracy: 0.6026 - val_loss: 0.5445 - val_accuracy: 0.7274 - lr: 0.0050
Epoch 2/50
39/39 [==============================] - 2s 40ms/step - loss: 0.4034 - accuracy: 0.8131 - val_loss: 0.2251 - val_accuracy: 0.9177 - lr: 0.0050
Epoch 3/50
39/39 [==============================] - 2s 40ms/step - loss: 0.2042 - accuracy: 0.9292 - val_loss: 0.1972 - val_accuracy: 0.9257 - lr: 0.0050
Epoch 4/50
39/39 [==============================] - 2s 41ms/step - loss: 0.1404 - accuracy: 0.9529 - val_loss: 0.1784 - val_accuracy: 0.9355 - lr: 0.0050
Epoch 5/50
39/39 [==============================] - 1s 39ms/step - loss: 0.1066 - accuracy: 0.9652 - val_loss: 0.1877 - val_accuracy: 0.9355 - lr: 0.0050
Epoch 6/50
39/39 [==============================] - 1s 38ms/step - loss: 0.0876 - accuracy: 0.9732 - val_loss: 0.1918 - val_accuracy: 0.9257 - lr: 0.0050
Epoch 7/50
39/39 [==============================] - 1s 37ms/step - loss: 0.0

In [32]:
model.save("LSTM_balanced.h5")

In [33]:
accr = model.evaluate(test_sequences_matrix,Y_test)

51/51 [==============================] - 1s 14ms/step - loss: 0.2069 - accuracy: 0.9288


In [34]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.207
  Accuracy: 0.929


In [35]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred)

51/51 [==============================] - 1s 13ms/step
[[0.9894442 ]
 [0.98014796]
 [0.00666841]
 ...
 [0.9901635 ]
 [0.0073009 ]
 [0.99572647]]


In [36]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

In [37]:
print(classification_report(Y_test, y_pred, digits=6))

              precision    recall  f1-score   support

           0   0.922612  0.936196  0.929354       815
           1   0.935162  0.921376  0.928218       814

    accuracy                       0.928791      1629
   macro avg   0.928887  0.928786  0.928786      1629
weighted avg   0.928883  0.928791  0.928786      1629

